In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
file_root = 'Reverse_Logistics_Template.xlsx'
tender = pd.read_excel(file_root)
tender.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_Port,Destination_Port,Origin_City,Destination_City,Equipment_Type,Shipping_Terms,Commodity,Volume
0,CustomerB,1089,ID,CN,Makassar,Xingang/Tianjin,Makassar,Xingang,20' Standard Dry,Port to Port,Cocoa Liquor,60.0
1,CustomerB,1090,ID,CN,Makassar,Shanghai,Makassar,Shanghai,20' Standard Dry,Port to Port,Cocoa Liquor,140.0
2,CustomerB,1093,CN,TH,Shanghai,Bangkok (Pat/Lat Krabang/Laem Chabang),Shanghai,Bangkok,20' Reefer,Port to Port,Confectionary,10.0
3,CustomerB,1094,CN,TH,Shanghai,Bangkok (Pat/Lat Krabang/Laem Chabang),Shanghai,Bangkok,40' Reefer,Port to Port,Confectionary,70.0
4,CustomerB,1095,CN,VN,Shanghai,Cat Lai,Shanghai,Ho Chi Minh,20' Reefer,Port to Port,Confectionary,5.0


In [4]:
#Let's split the Tender DataFrame in one dataframe for each customer
unique_customers = []
for customer in tender['Customer'].unique():
    locals()[customer] = tender[(tender.Customer == customer)]
    unique_customers.append(customer)
print(unique_customers)


['CustomerB', 'CustomerA']


In [5]:
#Let's focus on Customer A and see in which ports we could applye Reverse Logistics
CustomerA.describe()

,Volume
count,1134.000000
mean,145.579365
std,547.330020
min,1.000000
25%,10.000000
50%,12.000000
75%,48.000000
max,8557.000000


In [6]:
CustomerA_clean = CustomerA[CustomerA['Origin_Port'].notnull()]
CustomerA_clean.describe()

,Volume
count,855.000000
mean,173.614035
std,615.548097
min,1.000000
25%,10.000000
50%,12.000000
75%,59.500000
max,8557.000000


In [7]:
print('Before', CustomerA['Volume'].sum())
print('After', CustomerA_clean['Volume'].sum())

Before 165087.0
After 148440.0


In [8]:
Volume_Lost = CustomerA['Volume'].sum() - CustomerA_clean['Volume'].sum()
Volume_Lost

16647.0

In [9]:
CustomerA_Port_Exports = CustomerA.groupby(['Origin_Port']).sum()
CustomerA_Port_Exports = CustomerA_Port_Exports.rename({'Volume': 'Export_Volume'}, axis=1)
CustomerA_Port_Exports.head()

,Export_Volume
Origin_Port,
Alexandria,1872.0
Antwerp,28401.0
Barcelona,197.0
Bibao,12.0
Bremerhaven,4644.0


In [10]:
CustomerA_Port_Imports = CustomerA.groupby(['Destination_Port']).sum()
CustomerA_Port_Imports = CustomerA_Port_Imports.rename({'Volume': 'Import_Volume'}, axis=1)
CustomerA_Port_Imports.head()

,Import_Volume
Destination_Port,
Abidjan,60.0
Acajutla,2110.0
Ad Dammam,285.0
Adelaide,513.0
Ajman,1079.0


In [11]:
#Let's concatenate Origin and destination ports in order to see the Ports with higher volume, then we will focuse on Cities and Equipment types

In [12]:
total_by_port = pd.concat([CustomerA_Port_Exports, CustomerA_Port_Imports], axis=1, join='inner')
total_by_port.reset_index(level=0, inplace=True)
total_by_port

,index,Export_Volume,Import_Volume
0,Antwerp,28401.0,842.0
1,Barcelona,197.0,748.0
2,Bremerhaven,4644.0,76.0
3,Buenaventura,1029.0,2307.0
4,Busan,12.0,3762.0
5,Callao,60.0,5036.0
6,Cape Town,117.0,184.0
7,Cartagena,2844.0,701.0
8,Caucedo,767.0,1904.0
9,Copenhagen,63.0,131.0


In [17]:
selected_port = input('Select Port')
print(selected_port)


Select Port Melbourne


Melbourne


In [18]:
reverse_Origin_Port = CustomerA[CustomerA['Origin_Port']==selected_port]
reverse_Origin_Port

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_Port,Destination_Port,Origin_City,Destination_City,Equipment_Type,Shipping_Terms,Commodity,Volume


In [19]:
reverse_Origin_Port = CustomerA[CustomerA['Origin_Port']==selected_port]
reverse_Origin_Port = reverse_Origin_Port.groupby(['Origin_City', 'Equipment_Type'])['Volume'].sum()
reverse_Origin_Port

Series([], Name: Volume, dtype: float64)

In [20]:
reverse_Origin_Port = CustomerA[CustomerA['Origin_Port']==selected_port]
reverse_Origin_Port = reverse_Origin_Port.groupby(['Origin_City', 'Equipment_Type']).agg({'Volume': ['sum']})
reverse_Origin_Port

,,Volume
,,sum
Origin_City,Equipment_Type,


In [21]:
#For Customer A, we see that if customers gives us the city, they don't give us the Port.
#In this case, we should focuse on a port level. 
#Let's now focuse on Customer B
CustomerB.describe()

,Volume
count,1403.000000
mean,47.051465
std,136.227550
min,0.000000
25%,5.000000
50%,12.000000
75%,35.000000
max,2088.000000


In [22]:
CustomerB_clean = CustomerB[CustomerB['Origin_Port'].notnull()]
CustomerB_clean.describe()

,Volume
count,1380.000000
mean,47.602323
std,137.283928
min,0.000000
25%,5.000000
50%,12.000000
75%,35.000000
max,2088.000000


In [23]:
print('Before', CustomerB['Volume'].sum())
print('After', CustomerB_clean['Volume'].sum())

Before 66013.20506353732
After 65691.20506353732


In [24]:
Volume_Lost = CustomerB['Volume'].sum() - CustomerB_clean['Volume'].sum()
Volume_Lost

322.0

In [25]:
CustomerB_Port_Exports = CustomerB.groupby(['Origin_Port']).sum()
CustomerB_Port_Exports = CustomerB_Port_Exports.rename({'Volume': 'Export_Volume'}, axis=1)
CustomerB_Port_Exports.head()

,Export_Volume
Origin_Port,
Aarhus,15.0
Algeciras,240.0
Altamira,37.0
Ambarli,9.0
Ancona,20.0


In [26]:
CustomerB_Port_Imports = CustomerB.groupby(['Destination_Port']).sum()
CustomerB_Port_Imports = CustomerB_Port_Imports.rename({'Volume': 'Import_Volume'}, axis=1)
CustomerB_Port_Imports.head()

,Import_Volume
Destination_Port,
AL Khoms,8.0
Abu Dhabi,22.0
Al Hidd,15.0
Al Khoms,38.0
Al-Khums,65.0


In [27]:
total_by_port = pd.concat([CustomerB_Port_Exports, CustomerB_Port_Imports], axis=1, join='inner')
total_by_port.reset_index(level=0, inplace=True)
total_by_port

,index,Export_Volume,Import_Volume
0,Antwerp,1015.000000,672.030238
1,Antwerp / Rotterdam,7.000000,504.000000
2,Aqaba,10.000000,236.208235
3,Bangkok (Lat Krabang),30.000000,430.000000
4,Bremerhaven,763.000000,91.000000
5,Brisbane,39.000000,374.000000
6,Busan,386.000000,1824.734000
7,Colombo,120.000000,86.343591
8,Davao,408.000000,119.000000
9,Durban,1076.000000,1179.520833


In [31]:
selected_port = input('Select Port')
print(selected_port)

Select Port Melbourne


Melbourne


In [32]:
reverse_Origin_Port = CustomerB[CustomerB['Origin_Port']==selected_port]
reverse_Origin_Port.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_Port,Destination_Port,Origin_City,Destination_City,Equipment_Type,Shipping_Terms,Commodity,Volume
279,CustomerB,1616,AU,TH,Melbourne,Bangkok (Lat Krabang),Melbourne,Bangkok (Lat Krabang),20' Standard Dry,Port to Port,Petfood,10.0
280,CustomerB,1617,AU,VN,Melbourne,Ho Chi Minh,Melbourne,Ho Chi Minh,20' Standard Dry,Port to Port,Petfood,9.0
281,CustomerB,1618,AU,TH,Melbourne,Bangkok (Lat Krabang),Melbourne,Bangkok (Lat Krabang),40' High Cube,Port to Port,Petfood,100.0
282,CustomerB,1619,AU,HK,Melbourne,Hong Kong,Melbourne,Hong Kong,20' Standard Dry,Port to Port,Petfood,5.0
283,CustomerB,1620,AU,JP,Melbourne,Hakata,Melbourne,Hakata,20' Standard Dry,Port to Port,Petfood,2.0


In [33]:
reverse_Origin_Port = CustomerB[CustomerB['Origin_Port']==selected_port]
reverse_Origin_Port['City_Equipment'] = reverse_Origin_Port['Origin_City'] + reverse_Origin_Port['Equipment_Type']
reverse_Origin_Port = reverse_Origin_Port.groupby('City_Equipment').sum()
reverse_Origin_Port = reverse_Origin_Port.rename({'Volume': 'Origin_Volume'}, axis=1)
reverse_Origin_Port.sort_values('Origin_Volume', ascending=False)


/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Origin_Volume
City_Equipment,
Melbourne40' High Cube,989.0
Melbourne40' Reefer,803.0
Melbourne20' Standard Dry,612.0
Melbourne20' Reefer,152.0
Melbourne 40' Reefer,4.0


In [34]:
#Let's do the same for Imports and concatenate it
reverse_Destination_Port = CustomerB[CustomerB['Destination_Port']==selected_port]
reverse_Destination_Port['City_Equipment'] = reverse_Destination_Port['Destination_City'] + reverse_Destination_Port['Equipment_Type']
reverse_Destination_Port = reverse_Destination_Port.groupby('City_Equipment').sum()
reverse_Destination_Port = reverse_Destination_Port.rename({'Volume': 'Destination_Volume'}, axis=1)
reverse_Destination_Port.sort_values('Destination_Volume', ascending=False)


/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Destination_Volume
City_Equipment,
Melbourne40' High Cube,3352.090173
Melbourne40' Reefer,859.441412
Melbourne20' Standard Dry,72.000000
WOD40' Reefer,60.000000
Melbourne20' Reefer,39.000000
WOD20' Reefer,13.000000


In [35]:
reverse_total_port = pd.concat([reverse_Origin_Port, reverse_Destination_Port], axis=1, join='outer')
reverse_total_port

/home/dsc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Origin_Volume,Destination_Volume
Melbourne 40' Reefer,4.0,NaN
Melbourne20' Reefer,152.0,39.000000
Melbourne20' Standard Dry,612.0,72.000000
Melbourne40' High Cube,989.0,3352.090173
Melbourne40' Reefer,803.0,859.441412
WOD20' Reefer,NaN,13.000000
WOD40' Reefer,NaN,60.000000


In this case, we see that if we select Melborune, there are some interesting coincidence. Mainly Melbourne with 40'High Cube were we see 989 containers going to Melbourne city to the port, and 3352 containers going to the port to the city. This example would work for reverse logistics.

The next step would be to match Nan Destination Volume with Origin, for this, I will use Google Maps Api and see for example if there is a short distance between Melbourne and WOD. 

## How should the Front end look like?

After discussing with my company's sales managers, the following is the format they would like to see

In [55]:
front_end_destination = CustomerB[(CustomerB['Destination_Port']=='Melbourne') & (CustomerB['Equipment_Type']=="40' High Cube")]
front_end_destination = front_end_destination[['Lane_ID', 'Origin_Country', 'Origin_Port', 'Volume' ]]
front_end_destination.head()

,Lane_ID,Origin_Country,Origin_Port,Volume
133,1291,TH,Lat Krabang (via LCB),1098.090173
248,1480,LT,Klaipeda,40.000000
250,1482,HU,Koper,600.000000
265,1532,US,Memphis CY via LA / Long Beach,52.000000
579,2077,AT,SIKOP - DEHAM - DEBRV,240.000000


In [53]:
front_end_origin = CustomerB[(CustomerB['Origin_Port']=='Melbourne') & (CustomerB['Equipment_Type']=="40' High Cube")]
front_end_origin = front_end_origin[['Lane_ID', 'Destination_Country', 'Destination_Port', 'Volume' ]]
front_end_origin.head()

,Lane_ID,Destination_Country,Destination_Port,Volume
281,1618,TH,Bangkok (Lat Krabang),100.0
284,1621,JP,Hakata,52.0
285,1622,HK,Hong Kong,30.0
287,1624,ID,Jakarta,1.0
289,1626,PH,Manila,1.0


The first table (Front_end_destination) shows us the relevant information we need for reverse logistics from an origin perspective. All the lanes in this table
have Melbourne with 40' High Cubes as destination. The table below (Front_end_origin) shows as the other way around; Trade lanes from Melbourne with 40' High Cubes.
This format helps the user to identify potential opportunities, for example in this case, we can easily see that we have 1.098 containers from Thailand to Melbourne, and in the table below we have 100 containers from Melbourne to Thailandia. This would be a perfect example for reverse logistics, here we could safe transport costs and CO2 emissions for 100 containers in Melbourne. Also, since the port in Thailand matches, it would be interessting to see if we can also match these containers in Thailand.